In [841]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model

df = pd.read_csv("GroupExam2024_data_and_documentation/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", sep=';', decimal=',')


In [842]:
#print(df.info())
pd.set_option('display.max_rows', None)
print(df['Hovedart FAO'].value_counts())

Hovedart FAO
Torsk                            86215
Sei                              66311
Dypvannsreke                     36977
Hyse                             28176
Lange                            13387
Øyepål                            7796
Brosme                            7168
Sild                              6846
Kolmule                           6166
Snøkrabbe                         6070
Snabeluer                         6065
Lysing                            5964
Blåkveite                         4986
Breiflabb                         2552
Tobis og annen sil                2064
Lodde                             1378
Hestmakrell                       1224
Makrell                           1146
Vassild                           1120
Flekksteinbit                      896
Uer (vanlig)                       785
Lyr                                780
Stortare                           683
Rødspette                          615
Antarktisk krill                   552
Brisling    

In [843]:
print(df['Art FAO'].value_counts())

Art FAO
Torsk                            56574
Sei                              42557
Hyse                             39120
Lange                            17338
Uer (vanlig)                     14153
Dypvannsreke                     13678
Brosme                            9167
Lysing                            8839
Breiflabb                         8543
Blåkveite                         8046
Flekksteinbit                     8037
Snøkrabbe                         6070
Sild                              6006
Kveite                            5649
Snabeluer                         5408
Gråsteinbit                       5389
Lyr                               4863
Makrell                           3013
Smørflyndre                       2869
Kolmule                           2843
Annen marin fisk                  2378
Øyepål                            2210
Rødspette                         2206
Hvitting                          2175
Annen skate og rokke              2136
Blåsteinbit      

In [844]:
# Reset display options to default
pd.reset_option('display.max_rows')

x = df['Bruttotonnasje annen']
df['Bruttotonnasje 1969'].fillna(x, inplace=True) # Slår sammen kolonnene
df.drop(columns = ['Bruttotonnasje annen'], inplace=True) # Dropper Bruttotonnasje annen.
df.dropna(inplace = True)

df.drop_duplicates(inplace = True) # Dropper duplikater
df['Art - FDIR'].value_counts()


Art - FDIR
Torsk                  55749
Sei                    41903
Hyse                   38735
Lange                  17040
Uer (vanlig)           14108
                       ...  
Blåhval                    1
Annen makrell              1
Ansjos                     1
Rød kråkebolle             1
Bukstripet pelamide        1
Name: count, Length: 117, dtype: int64

In [845]:
def join_unique_values(series):
    return ', '.join(series.dropna().astype(str).unique())

# Adjusting the grouping to consider both Melding ID and an identifier for fishing activities, assuming 'Starttidspunkt' is that identifier.
df_grouped = df.groupby(['Melding ID', 'Starttidspunkt'], as_index=False).agg({
    'Startposisjon bredde': 'first',
    'Startposisjon lengde': 'first',
    'Startklokkeslett': 'first',
    'Havdybde start': 'first',
    'Stopposisjon bredde': 'last',
    'Stopposisjon lengde': 'last',
    'Havdybde stopp': 'last',
    'Trekkavstand': 'first',  # Adjusted based on assumption that distance is consistent within an activity.
    'Redskap FDIR': join_unique_values,
    'Hovedart FAO': 'first',  # Assuming main species is consistent within an activity.
    'Art - FDIR': join_unique_values,
    'Art - gruppe': join_unique_values,
    'Rundvekt': 'sum',  # Summing up the total weight.
    'Lengdegruppe (kode)': 'first',  
    'Bruttotonnasje 1969': 'first'
})
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91185 entries, 0 to 91184
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Melding ID            91185 non-null  int64  
 1   Starttidspunkt        91185 non-null  object 
 2   Startposisjon bredde  91185 non-null  float64
 3   Startposisjon lengde  91185 non-null  float64
 4   Startklokkeslett      91185 non-null  object 
 5   Havdybde start        91185 non-null  int64  
 6   Stopposisjon bredde   91185 non-null  float64
 7   Stopposisjon lengde   91185 non-null  float64
 8   Havdybde stopp        91185 non-null  int64  
 9   Trekkavstand          91185 non-null  float64
 10  Redskap FDIR          91185 non-null  object 
 11  Hovedart FAO          91185 non-null  object 
 12  Art - FDIR            91185 non-null  object 
 13  Art - gruppe          91185 non-null  object 
 14  Rundvekt              91185 non-null  float64
 15  Lengdegruppe (kode)

In [846]:
#Konverterer til datetime
df_grouped['Startklokkeslett'] = pd.to_datetime(df_grouped['Startklokkeslett'], format='%H:%M')
bins = list(range(0, 25))
labels = list(range(24))
df_grouped['Startklokkeslett - bins'] = pd.cut(df_grouped['Startklokkeslett'].dt.hour, bins=bins, labels=labels, right=False)
df_grouped['Startklokkeslett - bins'].value_counts()
df_grouped['Startklokkeslett - bins'].value_counts().sort_index()

Startklokkeslett - bins
0     5585
1     2855
2     2873
3     3832
4     4260
5     4734
6     4718
7     4285
8     4016
9     3894
10    3942
11    4212
12    4315
13    4052
14    3693
15    3531
16    3491
17    3677
18    3421
19    3007
20    2976
21    3085
22    3443
23    3288
Name: count, dtype: int64

Kosinus time.

In [847]:
fish_types = ['Hyse', 'Torsk', 'Sei', 'Lange']
fish_tools = ['Bunntrål', 'Dobbeltrål', 'Snurrevad', 'Udefinert trål']

condition_fish_types = df_grouped['Hovedart FAO'].isin(fish_types)
condition_fish_tools = df_grouped['Redskap FDIR'].isin(fish_tools)

combined_condition = condition_fish_tools & condition_fish_types # & condition_areas 
filtered_df = df[combined_condition]
filtered_df.info()

/var/folders/87/czlwvtcd4j3cz6jlhc_45ld00000gn/T/ipykernel_30072/2006037163.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = df[combined_condition]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
filtered_df['Startklokkeslett - bins'].value_counts().sort_index()

KeyError: 'Startklokkeslett - bins'

In [ ]:
# Calculate the counts of each unique value in 'Hovedart - FDIR (kode)'
counts = filtered_df['Hovedart FAO'].map(filtered_df['Hovedart FAO'].value_counts())

counts

1         18846
2         18846
3         18846
4         18846
5         18846
          ...  
305429    18846
305430    18846
305431    18846
305432    18846
305433    18846
Name: Hovedart FAO, Length: 137240, dtype: int64

In [ ]:
filtered_df['Hovedart FAO'].value_counts()

Hovedart FAO
Torsk    61354
Sei      55862
Hyse     18846
Lange     1178
Name: count, dtype: int64